In [301]:
import sqlite3
import json
from os import getenv
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pytesseract
from PIL import Image
import cv2
import numpy as np

In [302]:
USERNAME = getenv("USERNAME")
PASSWORD = getenv("PASSWORD")

In [349]:
chrome_options = Options()
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--enable-cookies")
url = "https://manganato.com/"

driver = webdriver.Chrome(
    "../../chromedriver-win64/chromedriver.exe",
    options=chrome_options
)

with open("manga_cookies.json", "r") as file:
    cookies = json.load(file)

driver.get(url)

for cookie in cookies:
    driver.add_cookie(cookie)

driver.refresh()

In [330]:
element = driver.find_element_by_xpath("//a[contains(@class, 'btn-login')]")
element.click()

user_name = driver.find_element_by_xpath("//input[contains(@class, 'user-name')]")
user_name.send_keys(USERNAME)

password = driver.find_element_by_xpath("//input[contains(@class, 'pass')]")
password.send_keys(PASSWORD)



In [331]:
print(driver.current_url)

https://manganato.com/


In [332]:
with open("manga_cookies.json", "w") as file:
    json.dump(driver.get_cookies(), file)

In [350]:
driver.close()

In [351]:
with open("manga_cookies.json", "r") as file:
    cookies = json.load(file)

for cookie in cookies:
    # if "expiry" in cookie:
    #     del cookie["expiry"]
    print(cookie)

{'domain': '.manganato.com', 'httpOnly': False, 'name': '_ga', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'GA1.1.934912023.1717840273'}
{'domain': 'manganato.com', 'httpOnly': False, 'name': 'user_acc', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': '%7B%22user_version%22%3A%222.3%22%2C%22user_name%22%3A%22Wallflower%22%2C%22user_image%22%3A%22https%3A%5C%2F%5C%2Favt.mkklcdnv6temp.com%5C%2Fcontent%5C%2Favt.png%22%2C%22user_data%22%3A%22eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.clNXdfJGbnJyeGbiojIlBXe09lJGbnJCLiwWYj96ICZp9lclNXdfLiYDN2MDO0EjIlclNXdfJGbnJClGZiojIyV2c19sIyYpB3b0VHdzXyV2c19lYsdmI2FwaSI6IiJ9.iXtuQrzMwtDCNPFQbDsY9a0y3pR2VpyVi98ICODkgu4%22%7D'}
{'domain': '.manganato.com', 'httpOnly': False, 'name': '_ga_KB8FZMQYTR', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'GS1.1.1717840273.1.0.1717840273.0.0.0'}
{'domain': '.manganato.com', 'httpOnly': False, 'name': '_gat_gtag_UA_198007309_1', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': '